In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 38.5 MB/s eta 0:00:00


In [ ]:
import os
from transformers import pipeline
import rdflib
from rdflib.namespace import RDFS, FOAF, RDF, OWL
from rdflib import Graph, URIRef, Literal
import numpy as np
from sentence_transformers import SentenceTransformer
import csv
import json
from huggingface_hub import login

In [ ]:
sentences_path = "/content/drive/MyDrive/FINKI/WBS, IKT KG Project/data/sentences/sentences.json"

In [ ]:
with open(sentences_path) as f1:
    sentences = json.load(f1)

In [ ]:
embeddings_file_path = "/content/drive/MyDrive/FINKI/WBS, IKT KG Project/data/embeddings.npy"
snomed_ontology_file_path = "/content/drive/MyDrive/FINKI/WBS, IKT KG Project/data/snomed_cf.owl"
labels_file_path = "/content/drive/MyDrive/FINKI/WBS, IKT KG Project/data/labels.json"
triplet_output_path = "/content/drive/MyDrive/FINKI/WBS, IKT KG Project/data/triplets/triplets.csv"

In [ ]:
embeddings = np.load(embeddings_file_path)

In [ ]:
with open(labels_file_path, 'r') as f:
    labels = json.load(f)

In [ ]:
pipe = pipeline("text-generation", model="lastmass/Qwen3_Medical_GRPO")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/649 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
model = SentenceTransformer("abhinand/MedEmbed-large-v0.1")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
def get_classes(sen: str, top_k=20):
    # Encode the input sentence
    sen_emb = model.encode(sen)

    # Normalize embeddings
    sen_emb_norm = sen_emb / np.linalg.norm(sen_emb)
    embeddings_norm = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    # Compute cosine similarities
    similarities = np.dot(embeddings_norm, sen_emb_norm)

    # Get top-k indices
    top_k_indices = np.argsort(similarities)[::-1][:top_k]

    # Retrieve corresponding labels
    classes = [labels[i] for i in top_k_indices]
    return classes

In [ ]:
get_classes("adapted breathing exercises")

['Adopts particular posture for breathing',
 'Breathing easily',
 'Deep breathing',
 'Stertorous breathing',
 'Ineffective breathing pattern',
 'Diaphragmatic breathing',
 'Amphoric breathing',
 'High altitude periodic breathing',
 'O/E - breathing method',
 'Thoracic breathing',
 'Shallow breathing',
 'O/E - stertorous breathing',
 'Slow shallow breathing',
 'Irregular breathing',
 'Abnormal breathing',
 'Able to take deep breaths',
 'Finding related to ability to take deep breaths',
 'Tracheal breathing',
 'Interrupted breathing',
 'Mouth breathing']

In [ ]:
system_message = f"""You are a powerfull model for creating triplets and giving a simple and short working out."""

In [ ]:
def persist_triplets(triplets):
  with open(triplet_output_path, "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(triplets)

In [ ]:
def extract_triplets(results, patient_name):
  triplets = []
  for result in results:
    triplet = []
    for word in result.split("<SOLUTION>")[1].split("</SOLUTION>")[0].split("'"):
      if word.strip() != '':
        triplet.append(word)
    if len(tuple(triplet)) == 3:
      triplet[0] = patient_name
      triplets.append(tuple(triplet))
  return triplets

In [ ]:
def generate_triplets(sentences, report, patient_name, print_results=False):
  results = []
  for sen in sentences:
    classes = get_classes(sen)

    human_message = f"""You need to create a triplet based on a sentence and some relevant classes.
Pick the most relevant class and create a triplet. Never make up information, use only from the context and the given sentence.
Never make up new classes, use only the given ones. If no class is relevant return <SOLUTION>None</SOLUTION>

Think about the problem and provide a short and simple working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your triplet between <SOLUTION></SOLUTION>

Example response 1:
Sentence:
male patient
Classes:
['Female', 'Age more than 50 years, male', 'Male', 'Well male adult', 'Age 60 to 64 years']
<start_working_out>Based on the text the patient appears to be male<end_working_out>
<SOLUTION>'Patient 2' 'is' 'Male'</SOLUTION>

Example respone 2:
Sentence:
wrist pain from motor vehicle accident
Classes:
['Pain in wrist', 'Injury of wrist', 'Other wrist injuries', 'Wrist joint tender', 'Wrist joint pain']
<start_working_out>The patient had some pain in the wrist from the motor accident.<end_working_out>
<SOLUTION>'Patient 2' 'has' 'Pain in wrist'</SOLUTION>

Example response 3:
Sentence:
positive COVID-19 result
Classes:
['Schick test positive', 'Pregnancy test positive', 'HIV positive', 'Hepatitis A test positive']
<start_working_out>The relevant class is not listed. None of the classes are relevant.<end_working_out>
<SOLUTION>None</SOLUTION>

Begin!
Given the context for patient '{patient_name}':
{report}
Sentence:
{sen}

Classes:
{classes}"""

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": human_message}
    ]
    result = pipe(messages, do_sample=False)
    if print_results:
      print(sen)
      print(result[0]["generated_text"][2]["content"])
      print(classes)
      print()
    results.append(result[0]["generated_text"][2]["content"])
  return results

In [ ]:
starting_index = 0
for index, sen in enumerate(sentences):
  if index >= starting_index:
    patient_name = f"Patient {sen['patient_id']}"
    report = sen["report"]
    sentence = sen["sentences"]
    results = generate_triplets(sentence, report, patient_name, print_results=False)
    triplets = extract_triplets(results, patient_name)
    persist_triplets(triplets)
    print("*"*30)
    print(f"Sentence {index} Finished")
    print("*"*30)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignore

******************************
Sentence 304 Finished
******************************


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignore

OutOfMemoryError: CUDA out of memory. Tried to allocate 354.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 152.12 MiB is free. Process 9750 has 14.59 GiB memory in use. Of the allocated memory 13.35 GiB is allocated by PyTorch, and 1.12 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)